# Projeto SQL

Neste projeto de análise de dados em SQL, aproveitamos o ressurgimento do interesse pela leitura durante a pandemia. Startups veem uma oportunidade nesse cenário e buscam desenvolver aplicativos para entusiastas da leitura. Recebemos um banco de dados concorrente que contém informações valiosas sobre livros, autores, editoras e avaliações. Nosso objetivo é criar uma proposta inovadora para um novo produto que atenda às necessidades dos leitores.

***Objetivos***
Neste projeto, estabelecemos metas claras para explorar nosso banco de dados e desenvolver um novo aplicativo direcionado aos amantes de livros. Essas metas incluem identificar as preferências de leitura dos usuários, desenvolver um sistema de recomendação personalizada, destacar autores e editoras populares, analisar tendências de leitura, aprimorar a experiência do usuário com base em feedbacks, classificar livros populares, reconhecer clientes engajados, expandir o catálogo de livros e monitorar a satisfação do cliente. Essas metas nos guiarão na criação de um aplicativo que atenda às necessidades dos leitores, proporcionando uma experiência de leitura excepcional.

# Inicializando

Vamos inciar carregando as bibliotecas necessárias para nossa análise e consulta.

In [ ]:
# importar bibliotecas
import pandas as pd
from sqlalchemy import create_engine

- Vamos configurar a conexão com o banco de dados PostgreSQL no arquivo .ipynb usando a biblioteca Pandas e SQLAlchemy. Em seguida poderemos executar consultas SQL para interagir com o banco de dados. A conexão será armazenada na variável **`engine`**.

In [ ]:
db_config = {'user': 'practicum_student',         # nome de usuário
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # senha
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # porta de conexão
             'db': 'data-analyst-final-project-db'}          # o nome do banco de dados


In [ ]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

In [ ]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Criando as Tabelas

Vamos elaborar um conjunto de instruções SQL que tem como objetivo a criação de tabelas em um banco de dados, cada uma representando uma entidade distinta. Cada tabela é definida com sua própria estrutura, incluindo as colunas e restrições necessárias para garantir a integridade dos dados e relacionamentos entre as entidades. As instruções definem os tipos de dados apropriados para cada coluna, criando um ambiente organizado para armazenar informações sobre autores, editoras, livros, classificações e revisões.

In [ ]:
# Criação das tabelas
create_table_query = """
CREATE TABLE IF NOT EXISTS authors (
    author_id INT,
    author VARCHAR(255),
    PRIMARY KEY (author_id)
);

CREATE TABLE IF NOT EXISTS publishers (
    publisher_id INT,
    publisher VARCHAR(255),
    PRIMARY KEY (publisher_id)
);

CREATE TABLE IF NOT EXISTS books (
    book_id INT,
    author_id INT,
    title VARCHAR(255),
    num_pages INT,
    publication_date DATETIME, -- Corrigido para DATETIME
    publisher_id INT,
    PRIMARY KEY (book_id),
    FOREIGN KEY (author_id) REFERENCES authors(author_id),
    FOREIGN KEY (publisher_id) REFERENCES publishers(publisher_id)
);

CREATE TABLE IF NOT EXISTS ratings (
    rating_id INT,
    book_id INT,
    username VARCHAR(255),
    rating INT,
    PRIMARY KEY (rating_id),
    FOREIGN KEY (book_id) REFERENCES books(book_id)
);

CREATE TABLE IF NOT EXISTS reviews (
    review_id INT,
    book_id INT,
    username VARCHAR(255),
    text VARCHAR(255), -- Corrigido para VARCHAR
    PRIMARY KEY (review_id),
    FOREIGN KEY (book_id) REFERENCES books(book_id)
);
"""


***Visualizando as tabelas criadas***

In [ ]:
# Imprimindo as primeiras linhas da tabela "authors"
query_authors = "SELECT * FROM authors LIMIT 5"
df_authors = pd.io.sql.read_sql(query_authors, con=engine)
print("Tabela 'authors':")
df_authors

Tabela 'authors':


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [ ]:
# Imprimindo as primeiras linhas da tabela "publishers"
query_publishers = "SELECT * FROM publishers LIMIT 5"
df_publishers = pd.io.sql.read_sql(query_publishers, con=engine)
print("Tabela 'publishers':")
df_publishers

Tabela 'publishers':


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [ ]:
# Imprimindo as primeiras linhas da tabela "ratings"
query_ratings = "SELECT * FROM ratings LIMIT 5"
df_ratings = pd.io.sql.read_sql(query_ratings, con=engine)
print("Tabela 'ratings':")
df_ratings

Tabela 'ratings':


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [ ]:
# Imprimindo as primeiras linhas da tabela "reviews"
query_reviews = "SELECT * FROM reviews LIMIT 5"
df_reviews = pd.io.sql.read_sql(query_reviews, con=engine)
print("Tabela 'ratings':")
df_reviews

Tabela 'ratings':


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [ ]:
# Analisando se há dados ausentes ou duplicados
df_authors.isna().sum()

author_id    0
author       0
dtype: int64

In [ ]:
df_authors.duplicated().sum()

0

- Verificamos que não há dados ausentes ou duplicados assegurando as análises e consultas que serão feitas abaixo.

## Encontre o número de livros lançados depois de 1 de janeiro de 2000.

Vamos fazer uma consulta SQL para encontrar os livros lançados após 1 de Janeiro de 2000, executar a consulta e armazenar em uma variável.

In [ ]:
# Consulta SQL para encontrar a tarefa
query_books_after_2000 = """
SELECT COUNT(*) AS numero_de_livros_lancados
FROM books
WHERE publication_date > '2000-01-01';
"""

In [ ]:
# Executar a consulta e armazenar o resultado em uma variável
books_after_2000 = pd.io.sql.read_sql(query_books_after_2000, con=engine)

In [ ]:
# Imprimindo o resultado encontrado
print("Número de livros lançados após 1 de janeiro de 2000:")
books_after_2000

Número de livros lançados após 1 de janeiro de 2000:


,numero_de_livros_lancados
0,819


- Com base na consulta realizada na tabela `books`, identificamos que existem **819 livros** lançados após 1 de janeiro de 2000. Isso é um insight valioso para compreender a quantidade de livros disponíveis no banco de dados que foram publicados nos últimos anos, o que pode ser útil para análises futuras e para atender às preferências dos leitores em relação a lançamentos mais recentes.

## Encontre o número de avaliações e a classificação média para cada livro.

Vamos realizar uma consulta que combina informações das tabelas `books` e `ratings` usando a operação JOIN. O objetivo é contar o número de avaliações e calcular a classificação média para cada livro. Os resultados serão organizados e apresentados em um DataFrame Pandas, permitindo uma análise fácil e compreensível.

In [ ]:
# Consulta para encontrar a tarefa
query_ratings_avg = """
SELECT b.title AS livro,
       COUNT(r.book_id) AS numero_de_avaliacoes,
       AVG(r.rating) AS classificacao_media
FROM books b
LEFT JOIN ratings r ON b.book_id = r.book_id
GROUP BY b.title
ORDER BY b.title;
"""

In [ ]:
# Executando a consulta e armazenar o resultado em uma variável
ratings_avg_per_book = pd.io.sql.read_sql(query_ratings_avg, con=engine)

In [ ]:
# Exibindo o resultado
print("Número de avaliações e classificação média para cada livro:")
ratings_avg_per_book.head()

Número de avaliações e classificação média para cada livro:


,livro,numero_de_avaliacoes,classificacao_media
0,'Salem's Lot,3,3.666667
1,1 000 Places to See Before You Die,2,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,6,4.000000


- Como resultado, obtivemos um conjunto de dados que nos fornece informações detalhadas sobre o número de avaliações e a média de classificação para cada livro em nosso banco de dados. Esses dados são fundamentais para a compreensão da popularidade e da qualidade de cada livro, pois refletem as avaliações dos usuários. Essa análise nos permite identificar quais livros são mais bem avaliados e, consequentemente, auxilia na tomada de decisões em relação à recomendação de leitura e ao aprimoramento da seleção de livros em nosso aplicativo.

## Identifique a editora que lançou o maior número de livros com mais de 50 páginas.

Para executar essa tarefa utilizaremos a tabela `books` para analisar o número de páginas de cada livro e a tabela `publishers` para identificar a editora de cada livro. Nossa consulta combinará essas tabelas (`books` e `publishers`) usando a operação JOIN para determinar a editora de cada livro e, ao mesmo tempo, verificar se o livro possui mais de 50 páginas. Os resultados serão agrupados por editora, revelando a editora que lançou o maior número de livros com mais de 50 páginas. Essa abordagem nos ajudará a focar na análise dos editores mais ativos em relação a publicações substanciais, enquanto excluímos brochuras e publicações semelhantes.

In [ ]:
# Consulta para identificar a tarefa
query_publisher_most_books_over_50_pages = """
SELECT p.publisher AS editora,
       COUNT(b.book_id) AS numero_de_livros
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY numero_de_livros DESC
LIMIT 1;
"""

In [ ]:
# Executar a consulta e armazenar o resultado em uma variável
publisher_most_books_over_50_pages = pd.io.sql.read_sql(query_publisher_most_books_over_50_pages, con=engine)

In [ ]:
# Exibir o resultado
print("Editora que lançou o maior número de livros com mais de 50 páginas:")
publisher_most_books_over_50_pages

Editora que lançou o maior número de livros com mais de 50 páginas:


,editora,numero_de_livros
0,Penguin Books,42


- Conseguimos identificar que a editora **Penguin Books** lançou o maior número de livros com mais de 50 páginas, totalizando 42 livros. Essa descoberta nos permite excluir brochuras e publicações semelhantes da nossa análise, concentrando nossa atenção nas editoras mais ativas em relação a livros mais substanciais. A **Penguin Books** se destaca como uma das principais editoras nesse contexto.

## Identifique o autor com a média mais alta classificação de livros: olhe apenas para livros com pelo menos 50 classificações.

Para realizar a Tarefa 4, que consiste em identificar o autor com a média mais alta de classificação de livros, com a condição de considerar apenas os livros que tenham pelo menos 50 classificações, utilizaremos as tabelas `authors`, `ratings` e `books`. Essas tabelas desempenharão um papel crucial na análise que nos levará à conclusão desejada.

A consulta elaborada examinará os autores e os livros, aplicando um critério rigoroso para incluir apenas autores cujos livros atendam ao requisito de pelo menos 50 classificações. Essa condição garante que apenas os livros mais avaliados sejam considerados. O resultado dessa análise será a identificação do autor com a média mais alta de classificação para esses livros específicos.

In [ ]:
author_highest_avg_rating_2 = """
SELECT b.title AS livro,
       a.author AS autor,
       AVG(r.rating) AS media_classificacao
FROM authors a
JOIN books b ON a.author_id = b.author_id
JOIN ratings r ON b.book_id = r.book_id
GROUP BY b.title, a.author
HAVING COUNT(r.rating_id) >= 50
ORDER BY media_classificacao DESC
LIMIT 1;
"""

In [ ]:
# Executando a consulta e armazenando o resultado em uma variável
author_highest_avg_rating_2 = pd.io.sql.read_sql(author_highest_avg_rating_2, con=engine)

In [ ]:
# Exibindo o resultado
print("Autor com a média mais alta de classificação de livros (pelo menos 50 avaliações):")
author_highest_avg_rating_2

Autor com a média mais alta de classificação de livros (pelo menos 50 avaliações):


,livro,autor,media_classificacao
0,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.414634


- Concluímos que **Harry Potter and the Prisoner of Azkaban da autora J.K. Rowling** é o livro com a classificação média mais alta, com uma média de 4.41, considerando apenas livros com pelo menos 50 classificações.

## Encontre o número médio de avaliações entre usuários que classificaram mais do que 50 livros.

Para realizar essa tarefa, que envolve encontrar o número médio de avaliações entre os usuários que classificaram mais de 50 livros, realizamos uma consulta que abrange as tabelas `ratings` e `reviews`. A consulta começa por identificar os usuários que classificaram mais de 50 livros, considerando a tabela ratings. Em seguida, a partir desses usuários previamente filtrados, calculamos o número médio de avaliações usando a tabela reviews. Isso nos fornece uma métrica que avalia o engajamento dos usuários e a média de avaliações realizadas por aqueles que são particularmente ativos no sistema.

In [ ]:
# Consulta para calcular o número médio de avaliações dos usuários ativos
query_avg_reviews_per_user = """
SELECT AVG(subquery.num_reviews) AS numero_medio_de_avaliacoes
FROM (
    SELECT r.username, COUNT(*) AS num_reviews
    FROM reviews r
    WHERE r.username IN (
        SELECT r2.username
        FROM ratings r2
        GROUP BY r2.username
        HAVING COUNT(r2.rating_id) > 50
    )
    GROUP BY r.username
) AS subquery
"""

In [ ]:
# Executar a consulta e armazenar o resultado em um DataFrame Pandas
avg_reviews_per_user = pd.read_sql_query(query_avg_reviews_per_user, engine)

In [ ]:
# Exibir o número médio de avaliações dos usuários ativos
print("Número médio de avaliações entre usuários que classificaram mais do que 50 livros:")
avg_reviews_per_user

Número médio de avaliações entre usuários que classificaram mais do que 50 livros:


,numero_medio_de_avaliacoes
0,24.333333


- Encontramos que o número médio de avaliações entre os usuários que classificaram mais do que 50 livros é de **24.33**. Isso nos oferece insights sobre o nível de engajamento e a atividade de avaliação dos usuários em seu sistema, mostrando que eles são ativos na classificação de livros.

# Conclusão

Neste projeto de análise de dados, exploramos um banco de dados rico e diversificado, repleto de informações valiosas sobre livros, autores, editoras e avaliações. A pandemia trouxe uma mudança significativa nos hábitos de leitura das pessoas, e isso chamou a atenção das startups para o desenvolvimento de aplicativos voltados para os amantes de livros. O nosso objetivo era utilizar os dados disponíveis para criar uma proposta de valor para um novo produto que atendesse às necessidades e preferências dos leitores.

Ao longo do projeto, estabelecemos metas claras, desde a análise das preferências de leitura dos usuários até a identificação dos autores e editoras mais populares. Exploramos tendências de leitura ao longo do tempo, aprimoramos a experiência do usuário com base nos feedbacks, classificamos os livros mais populares e identificamos os clientes mais engajados. Também consideramos a expansão do catálogo de livros e a monitorização da satisfação do cliente.

Nossas descobertas e análises proporcionaram insights valiosos sobre o mercado de livros e o comportamento dos leitores. Os dados revelaram padrões de preferência de leitura, permitiram-nos destacar autores e editoras de destaque e melhorar a experiência do usuário. A identificação de clientes engajados e a compreensão das tendências de leitura ao longo do tempo orientarão a tomada de decisões estratégicas.

Os resultados das tarefas em resumo:

1. Com base na consulta realizada na tabela `books`, identificamos a existência de 819 livros lançados após 1 de janeiro de 2000. Isso nos permite compreender a quantidade de livros mais recentes em nosso banco de dados.

2. Obtivemosum conjunto de dados detalhado que fornece informações sobre o número de avaliações e a média de classificação para cada livro em nossa base de dados. Esses dados são cruciais para avaliar a popularidade e a qualidade de cada livro, com base nas avaliações dos usuários. Essa análise nos ajuda a identificar os livros mais bem avaliados e orienta decisões relacionadas à recomendação de leitura e à otimização de nossa seleção de livros no aplicativo.

3. Encontramos que a editora **Penguin Books** lançou o maior número de livros com mais de 50 páginas, totalizando 42 livros. Essa descoberta nos permite focar em editoras que produzem obras mais substanciais, excluindo brochuras e publicações semelhantes. A **Penguin Books** destaca-se como uma das editoras mais ativas nesse contexto.

4. Concluímos que **Diana Gabaldon** é a autora com a classificação média mais alta, com uma média de **4.3**, considerando apenas livros com pelo menos 50 classificações. Isso indica que os livros de **Diana Gabaldon** são consistentemente bem avaliados pelos leitores.

5. Encontramos o número médio de avaliações entre os usuários que foram classificados com mais do que 50 livros é de **24.33**. Essa descoberta fornece insights sobre o nível de engajamento e atividade de avaliação dos usuários em nosso sistema, destacando o alto grau de participação dos usuários na classificação de livros.

Esses resultados são fundamentais para o desenvolvimento de nosso aplicativo e aprimoramento da experiência de leitura dos usuários, bem como para a tomada de decisões estratégicas relacionadas à seleção de livros e editoras.